In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import re
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [2]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ejfel\AppData\Roaming\nltk_data...


In [3]:
with open('coffee_words.pickle','rb') as read_file:
    coffee = pickle.load(read_file)
with open('coffee_ratings.pickle','rb') as read_file:
    ratings = pickle.load(read_file)
with open('combined.pickle','rb') as read_file:
    combined = pickle.load(read_file)
with open('df_full.pickle','rb') as read_file:
    df = pickle.load(read_file)
with open('df_topic_breakdown.pickle','rb') as read_file:
    df_topic_breakdown = pickle.load(read_file)
with open('sentiment.pickle','rb') as read_file:
    sentiment = pickle.load(read_file)

with open('blindtfidf_vec.pickle', 'rb') as read_file:
    blindtfidf = pickle.load(read_file)
with open('blindtfidf_mat.pickle', 'rb') as read_file:
    tfidf_blind = pickle.load(read_file)

In [28]:
sid = SentimentIntensityAnalyzer()
sentiment = pd.DataFrame()
sentiment['roaster'] = coffee.Roaster
sentiment['review'] = coffee.Review
sentiment['scores'] = coffee.Review.apply(lambda Text: sid.polarity_scores(Text))
sentiment['pos']  = df['scores'].apply(lambda score_dict: score_dict['pos'])
sentiment['neg']  = df['scores'].apply(lambda score_dict: score_dict['neg'])
sentiment['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
sentiment.drop(columns='scores',inplace=True)
# sentiment['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

In [30]:
with open('sentiment.pickle', 'wb') as to_write:
    pickle.dump(sentiment, to_write)
sentiment.sample(20)

,roaster,review,pos,neg,compound
4916,Hevla Coffee Company,Richly balanced aroma with hints of chocolate-...,0.067,0.120,-0.7140
5005,Surf City Coffee,For both Ken (90) and co-cupper Christy Thorns...,0.092,0.022,0.8316
3597,Wonderstate Coffee,"Ripe lemon dominates, with rounding complicati...",0.091,0.011,0.8779
1159,Roast House,"Floral-toned, crisp, deeply sweet. Dried straw...",0.113,0.017,0.8221
5296,Flying Goat Coffee,"Lindsey Bolger: ""Depending on tolerance for fr...",0.227,0.009,0.9913
27,Nostalgia Coffee Roasters,"Vibrantly sweet-tart, balanced. Lemon drop, r...",0.121,0.027,0.8074
4969,Quartermaine Coffee Roasters,Powerful and complex. Extraordinary aroma: spi...,0.192,0.000,0.9382
980,Yuban,Evaluated at proportions of 5 grams of instant...,0.043,0.080,-0.3923
5133,Stumptown Coffee Roasters,"Sweet and richly low-toned in the aroma, with ...",0.128,0.042,0.7506
5577,Heritage Coffee,"The cup is redolent with the fresh, sweet perf...",0.219,0.016,0.9783


In [9]:
df.comp_score.value_counts()

pos    5850
neg     109
Name: comp_score, dtype: int64

In [17]:
df[df.comp_score == 'neg']

,Roaster,Text,Two Topic,Three Topic,Four Topic,Five Topic,Six Topic,Seven Topic,Eight Topic,Ten Topic,Nine Topic,rating,scores,compound,comp_score
406,Cafe Grumpy,"Richly sweet-tart, earth-toned. Blood orange, ...",0,0,3,3,3,3,3,5,5,93,"{'neg': 0.06, 'neu': 0.902, 'pos': 0.038, 'com...",-0.5267,neg
495,Cameron’s Coffee,"Balanced, sweet-toned. Baking chocolate, almon...",1,1,1,1,1,1,1,1,1,89,"{'neg': 0.017, 'neu': 0.983, 'pos': 0.0, 'comp...",-0.0772,neg
978,Maxim Coffee,Evaluated at proportions of 5 grams of instant...,1,1,1,1,1,1,1,1,1,72,"{'neg': 0.057, 'neu': 0.901, 'pos': 0.042, 'co...",-0.2732,neg
980,Yuban,Evaluated at proportions of 5 grams of instant...,1,1,3,1,1,3,3,5,5,68,"{'neg': 0.08, 'neu': 0.877, 'pos': 0.043, 'com...",-0.3923,neg
1262,A.R.C.,"Evaluated as espresso. Fir, almond, baker’s ch...",1,1,1,1,1,1,1,1,1,88,"{'neg': 0.054, 'neu': 0.919, 'pos': 0.027, 'co...",-0.0516,neg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5910,Baja Vista,A powerful but flawed coffee. Coffee professio...,1,2,0,0,2,2,7,7,7,77,"{'neg': 0.117, 'neu': 0.81, 'pos': 0.073, 'com...",-0.8126,neg
5920,"Safeway, Inc.","For a moment, somewhere between nose and finis...",1,1,1,1,1,1,7,7,7,70,"{'neg': 0.167, 'neu': 0.747, 'pos': 0.086, 'co...",-0.8832,neg
5921,Lucky Stores,The vanilla / nut tones in the aroma fade quic...,1,1,1,1,1,1,7,7,7,68,"{'neg': 0.124, 'neu': 0.783, 'pos': 0.094, 'co...",-0.4404,neg
5923,Eight O'Clock Coffee,"Fermented notes, barely detectable in the acid...",1,1,1,1,2,2,7,7,7,60,"{'neg': 0.085, 'neu': 0.892, 'pos': 0.023, 'co...",-0.5494,neg


In [19]:
df.iloc[406].Text

'Richly sweet-tart, earth-toned. Blood orange, pipe tobacco, roasted almond, rhododendron, baking chocolate in aroma and cup. Tart-leaning structure with crisp, green-apple-like (malic) acidity; deep, velvety mouthfeel. The richly drying finish consolidates to blood orange and pipe  coffee tied for the fourth-highest rating in a cupping of “desert-island coffees” submitted by roasters for Coffee Review’s May 2020 tasting report. Muungano, the name of this cooperative, means “togetherness” in Swahili. One-third of the coop’s members are women, many of whom were widowed by war. Produced entirely from trees of strains of the Bourbon variety of Arabica and processed by the washed method (fruit skin and pulp are removed before drying). Certified organic. An integral part of New York’s coffee renaissance, Brooklyn-based Café Grumpy opened its first café in 2005 and has since expanded to 10 locations in New York, including a roasting facility, and one in Miami. '